In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['X_train.csv', 'sample_submission.csv', 'X_test.csv', 'y_train.csv']


In [34]:
test = pd.read_csv('../input/X_test.csv')
x_train = pd.read_csv('../input/X_train.csv')
y_train = pd.read_csv('../input/y_train.csv')

In [35]:
x_train.head(5)

,row_id,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
0,0_0,0,0,-0.75853,-0.63435,-0.10488,-0.10597,0.107650,0.017561,0.000767,-0.74857,2.1030,-9.7532
1,0_1,0,1,-0.75853,-0.63434,-0.10490,-0.10600,0.067851,0.029939,0.003385,0.33995,1.5064,-9.4128
2,0_2,0,2,-0.75853,-0.63435,-0.10492,-0.10597,0.007275,0.028934,-0.005978,-0.26429,1.5922,-8.7267
3,0_3,0,3,-0.75852,-0.63436,-0.10495,-0.10597,-0.013053,0.019448,-0.008974,0.42684,1.0993,-10.0960
4,0_4,0,4,-0.75852,-0.63435,-0.10495,-0.10596,0.005135,0.007652,0.005245,-0.50969,1.4689,-10.4410


In [36]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487680 entries, 0 to 487679
Data columns (total 13 columns):
row_id                   487680 non-null object
series_id                487680 non-null int64
measurement_number       487680 non-null int64
orientation_X            487680 non-null float64
orientation_Y            487680 non-null float64
orientation_Z            487680 non-null float64
orientation_W            487680 non-null float64
angular_velocity_X       487680 non-null float64
angular_velocity_Y       487680 non-null float64
angular_velocity_Z       487680 non-null float64
linear_acceleration_X    487680 non-null float64
linear_acceleration_Y    487680 non-null float64
linear_acceleration_Z    487680 non-null float64
dtypes: float64(10), int64(2), object(1)
memory usage: 48.4+ MB


In [37]:
y_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3810 entries, 0 to 3809
Data columns (total 3 columns):
series_id    3810 non-null int64
group_id     3810 non-null int64
surface      3810 non-null object
dtypes: int64(2), object(1)
memory usage: 89.4+ KB


In [38]:
y_train.head(5)

,series_id,group_id,surface
0,0,13,fine_concrete
1,1,31,concrete
2,2,20,concrete
3,3,31,concrete
4,4,22,soft_tiles


In [39]:
x_train.row_id.nunique()

487680

In [40]:
x_train.series_id.nunique()

3810

In [41]:
x_train.measurement_number.nunique()

128

In [ ]:
y_train.head(5)

In [ ]:
y_train.groupby('series_id').first()['surface'].value_counts()

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

plt.figure(figsize=(15,8))
sns.countplot( x = 'surface', data = y_train)

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot( x ='surface',y = 'group_id', data = y_train)

In [ ]:
plt.figure(figsize=(15,8))
sns.boxplot( x ='surface',y = 'series_id', data = y_train, palette = 'rainbow')

In [ ]:
fig, ax = plt.subplots(3, 3, figsize = (16, 12))
sns.distplot(x_train['angular_velocity_X'], kde= False, bins=30, ax=ax[0][0])
sns.distplot(x_train['angular_velocity_Y'], kde= False, bins=30, ax=ax[0][1])
sns.distplot(x_train['angular_velocity_Z'], kde= False, bins=30, ax=ax[0][2])
sns.distplot(x_train['linear_acceleration_X'], kde= False, bins=30, ax=ax[1][0])
sns.distplot(x_train['linear_acceleration_Y'], kde= False, bins=30, ax=ax[1][1])
sns.distplot(x_train['linear_acceleration_Z'], kde= False, bins=30, ax=ax[1][2])
sns.distplot(x_train['orientation_X'], kde= False, bins=30, ax=ax[2][0])
sns.distplot(x_train['orientation_Y'], kde= False, bins=30, ax=ax[2][1])
sns.distplot(x_train['orientation_Z'], kde= False, bins=30, ax=ax[2][2])

In [ ]:
x_train_2 = x_train.drop(['row_id','series_id', 'measurement_number'], axis=1)

In [ ]:
x_train_2 .head(5)

In [ ]:
sns.pairplot(x_train_2)

In [ ]:
#Function to calculate the norm of a three element vector so as to make a velocity column 
def vector_norm(x,y,z,df):
    return np.sqrt(df[x]**2 + df[y]**2 + df[z]**2)

#now combine all the x, y and z dimensions of velocity and acceleration to make 1 column 
x_train['angular_velocity_norm'] =vector_norm('angular_velocity_X',
                                                'angular_velocity_Y',
                                                'angular_velocity_Z',x_train)

x_train['linear_acceleration_norm'] =vector_norm('linear_acceleration_X',
                                                'linear_acceleration_Y',
                                                'linear_acceleration_Z',x_train)

#group by series id to get an avergae value of for every series id
new_x_train = x_train.groupby('series_id')['angular_velocity_norm','linear_acceleration_norm'].mean()
new_x_train.head(5)

In [ ]:
new_x_train  = pd.DataFrame(new_x_train).reset_index()
new_x_train.head(5)


In [ ]:
new_x_train.columns = ['serie_id','avg_velocity','avg_acceleration']
new_x_train['surface'] = y_train.surface
new_x_train['group_id'] = y_train.group_id

new_x_train.head(8)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (16, 12))
plt.figure(figsize=(15,8))
sns.boxplot( x ='surface',y = 'avg_velocity', data = new_x_train, palette = 'rainbow', ax=ax[0])
sns.boxplot( x ='surface',y = 'avg_acceleration', data = new_x_train, palette = 'rainbow', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (16, 12))
plt.figure(figsize=(15,8))
sns.barplot( x ='surface',y = 'avg_velocity', data = new_x_train, ax=ax[0])
sns.barplot( x ='surface',y = 'avg_acceleration', data = new_x_train, palette = 'rainbow', ax=ax[1])

In [ ]:
surfaces = new_x_train.surface.unique()
surfaces

In [ ]:
# we need to make swarmplot for every surface showing avg velocity by group id 
for surface in surfaces:
    sns.swarmplot(x=new_x_train[new_x_train.surface == surface]['group_id'],
                  y=new_x_train[new_x_train.surface == surface]['avg_velocity'])
    plt.title('Surface = {}'.format(surface))
    plt.show()

In [ ]:
# we need to make swarmplot for every surface showing avg velocity by group id 
for surface in surfaces:
    sns.swarmplot(x=new_x_train[new_x_train.surface == surface]['group_id'],
                  y=new_x_train[new_x_train.surface == surface]['avg_acceleration'])
    plt.title('Surface = {}'.format(surface))
    plt.show()

In [ ]:
surfaces = new_x_train.surface.unique()
for surface in surfaces:
    sns.jointplot(x= 'avg_velocity', y = 'avg_acceleration', data =new_x_train[new_x_train.surface == surface], kind = 'reg')
    plt.title('Surface = {}'.format(surface))
    plt.show()

In [ ]:
# pairwise correlation
x_train_2.corr(method='spearman').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)

In [ ]:
x_train_2.corr(method='pearson').style.format("{:.2}").background_gradient(cmap=plt.get_cmap('coolwarm'), axis=1)